In [4]:
from flask import Flask, request, jsonify
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import json



In [20]:
fillna_vals = pd.read_csv('fillna_vals.csv')
display(fillna_vals)

,cap-shape__bell,cap-shape__conical,cap-shape__convex,cap-shape__flat,cap-shape__knobbed,cap-shape__sunken,cap-surface__fibrous,cap-surface__grooves,cap-surface__scaly,cap-surface__smooth,...,population__scattered,population__several,population__solitary,habitat__grasses,habitat__leaves,habitat__meadows,habitat__paths,habitat__urban,habitat__waste,habitat__woods
0,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [3]:
with open('feat_dicts.json') as json_file:
    feat_dicts = json.load(json_file) 

In [12]:
feat_dicts

{'cap-shape': {'b': 'bell',
  'c': 'conical',
  'x': 'convex',
  'f': 'flat',
  'k': 'knobbed',
  's': 'sunken'},
 'cap-surface': {'f': 'fibrous', 'g': 'grooves', 'y': 'scaly', 's': 'smooth'},
 'cap-color': {'n': 'brown',
  'b': 'buff',
  'c': 'cinnamon',
  'g': 'gray',
  'r': 'green',
  'p': 'pink',
  'u': 'purple',
  'e': 'red',
  'w': 'white',
  'y': 'yellow'},
 'bruises': {'t': 'bruises', 'f': 'no'},
 'odor': {'a': 'almond',
  'l': 'anise',
  'c': 'creosote',
  'y': 'fishy',
  'f': 'foul',
  'm': 'musty',
  'n': 'none',
  'p': 'pungent',
  's': 'spicy'},
 'gill-attachment': {'a': 'attached',
  'd': 'descending',
  'f': 'free',
  'n': 'notched'},
 'gill-spacing': {'c': 'close', 'w': 'crowded', 'd': 'distant'},
 'gill-size': {'b': 'broad', 'n': 'narrow'},
 'gill-color': {'k': 'black',
  'n': 'brown',
  'b': 'buff',
  'h': 'chocolate',
  'g': 'gray',
  'r': 'green',
  'o': 'orange',
  'p': 'pink',
  'u': 'purple',
  'e': 'red',
  'w': 'white',
  'y': 'yellow'},
 'stalk-shape': {'e': '

In [11]:
kt = tuple(feat_dicts.keys())

('cap-shape',
 'cap-surface',
 'cap-color',
 'bruises',
 'odor',
 'gill-attachment',
 'gill-spacing',
 'gill-size',
 'gill-color',
 'stalk-shape',
 'stalk-root',
 'stalk-surface-above-ring',
 'stalk-surface-below-ring',
 'stalk-color-above-ring',
 'stalk-color-below-ring',
 'veil-type',
 'veil-color',
 'ring-number',
 'ring-type',
 'spore-print-color',
 'population',
 'habitat')

In [41]:
app = Flask(__name__)

model_from_disk = pickle.load(open('model.pkl','rb'))
ohe_from_disk = pickle.load(open('ohe.pkl','rb'))

@app.route('/')
def home():
    
    return '''Welcome
    Example url: http://127.0.0.1:5000/mushroom/?cap-shape=x&cap-surface=s&cap-color=n&bruises=t&odor=p&gill-attachment=f&gill-spacing=c&gill-size=n&gill-color=k&stalk-shape=e&stalk-root=e&stalk-surface-above-ring=s&stalk-surface-below-ring=s&stalk-color-above-ring=w&stalk-color-below-ring=w&veil-type=p&veil-color=w&ring-number=o&ring-type=p&spore-print-color=k&population=s&habitat=u
    '''

@app.route('/mushroom/')
def give_mushroom_predict():
    
    a = []
    cols=[]
    for i, (k, v) in enumerate(feat_dicts.items()):
        a.append(request.args.get(k))
        cols.append(k)
    
    # faire un dataframe avec la requête
    raw_input = pd.DataFrame(np.array(a).reshape(1,-1))
    raw_input.columns = cols

    print(raw_input)

    # traduire le dataframe selon onehotoncoder sur disque
    model_cols = model_from_disk.feature_names_in_
    new_data = pd.DataFrame(ohe_from_disk.transform(raw_input))
    new_data.columns = model_cols

    # faire une prediction
    #pred = model_from_disk.predict(fillna_vals)
    pred = model_from_disk.predict(new_data)
    
    # mettre en json le resultat de la prediction
    dict_pred = {'class':pred[0]}
    json_object = json.dumps(dict_pred, indent = 4) 

    return jsonify(dict_pred)

#http://127.0.0.1:5000/iris/?sepal_length=8&sepal_width=8&petal_length=8&petal_width=8

#http://127.0.0.1:5000/iris/?sepal_length=3&sepal_width=3&petal_length=0&petal_width=0

In [42]:
if __name__ == '__main__':

    app.run( port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jul/2023 14:57:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2023 14:57:30] "GET /mushroom/?cap-shape=x&cap-surface=s&cap-color=n&bruises=t&odor=p&gill-attachment=f&gill-spacing=c&gill-size=n&gill-color=k&stalk-shape=e&stalk-root=e&stalk-surface-above-ring=s&stalk-surface-below-ring=s&stalk-color-above-ring=w&stalk-color-below-ring=w&veil-type=p&veil-color=w&ring-number=o&ring-type=p&spore-print-color=k&population=s&habitat=u HTTP/1.1" 200 -


  cap-shape cap-surface cap-color bruises odor gill-attachment gill-spacing  \
0         x           s         n       t    p               f            c   

  gill-size gill-color stalk-shape  ... stalk-surface-below-ring  \
0         n          k           e  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  

[1 rows x 22 columns]
   cap-shape__bell  cap-shape__conical  cap-shape__convex  cap-shape__flat  \
0              0.0                 0.0                0.0              0.0   

   cap-shape__knobbed  cap-shape__sunken  cap-surface__fibrous  \
0                 0.0                1.0                   0.0   

   cap-surface__grooves  cap-surface__scaly  cap-surface__smooth  ...  \
0                   0.0                 1.0    

127.0.0.1 - - [27/Jul/2023 14:57:49] "GET /mushroom/?cap-shape=x&cap-surface=s&cap-color=n&bruises=t&odor=f&gill-attachment=f&gill-spacing=c&gill-size=n&gill-color=k&stalk-shape=e&stalk-root=e&stalk-surface-above-ring=s&stalk-surface-below-ring=s&stalk-color-above-ring=w&stalk-color-below-ring=w&veil-type=p&veil-color=w&ring-number=o&ring-type=p&spore-print-color=k&population=s&habitat=u HTTP/1.1" 200 -


  cap-shape cap-surface cap-color bruises odor gill-attachment gill-spacing  \
0         x           s         n       t    f               f            c   

  gill-size gill-color stalk-shape  ... stalk-surface-below-ring  \
0         n          k           e  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  

[1 rows x 22 columns]
   cap-shape__bell  cap-shape__conical  cap-shape__convex  cap-shape__flat  \
0              0.0                 0.0                0.0              0.0   

   cap-shape__knobbed  cap-shape__sunken  cap-surface__fibrous  \
0                 0.0                1.0                   0.0   

   cap-surface__grooves  cap-surface__scaly  cap-surface__smooth  ...  \
0                   0.0                 1.0    